In [ ]:
import os
import sys
%run ..//NewFunctions.py

#****************** fill these in *************************
base_dir = 'D:\\Segment'
project_name = 'K02288'
#****************** fill these in *************************

path = os.path.join(base_dir, project_name)
Path(os.path.join(path, 'seg', 'seeds', 'for_biowulf')).mkdir(parents=True, exist_ok=True)


files = []
for entry in os.scandir(os.path.join(path, 'seg', 'mem')):
    if entry.is_file():
        files.append(entry)

total_file_nums = len(files)


sys.path.append(os.path.abspath(os.path.join(base_dir, 'Scripts')))
shape = DDN.Utils.GetShape(path)
%gui qt

In [ ]:
#load all the images and stack them into a 4D array
membranes, seg, confidence, changes, average, small_areas, nuc, nuc_confidence, labels, labs_bin = DDN.Images.MakeStackedImages(path, min_tp = 54, max_tp = 56, membranes_bool=True, small_areas_bool=False, segmentation_bool=True, confidence_bool=False, changes_bool=False, average_bool=False, nuc_bool=False, nuc_intersect_bool=False, labels_bool=False, labs_bin_bool=True)
    

In [ ]:

seeds = np.load(path + '\\seg\\seeds\\array\\seeds_array.npy')


#overwrite the old seeds with these new ones
np.save(path + '\\seg\\seeds\\array\\old_seeds.npy', seeds)


viewer = napari.Viewer()



if type(membranes) == np.ndarray:
    viewer.add_image(membranes, name = 'membranes', blending='additive', colormap='green');

if type(seg) == np.ndarray:
    viewer.add_image(seg, name = 'seg', blending='additive', colormap='red');

if type(nuc) == np.ndarray:
    viewer.add_image(nuc, name = 'nuclei', blending='additive', colormap='gray');

if type(nuc_confidence) == np.ndarray:
    viewer.add_image(nuc_confidence, name = 'nuc confidence', blending='additive', colormap='yellow');

if type(small_areas) == np.ndarray:
    viewer.add_image(small_areas, name = 'small_areas', blending='additive', colormap='blue');

if type(confidence) == np.ndarray:
    viewer.add_image(confidence, name = 'potential_errors', blending='additive', colormap='gray');

if type(changes) == np.ndarray:
    viewer.add_image(changes, name = 'changes', blending='additive', colormap='blue');

if type(labels) == np.ndarray:
    viewer.add_image(labels, name = 'labels', blending='additive', colormap='blue');

if type(labs_bin) == np.ndarray:
    viewer.add_image(labs_bin, name = 'binary labels', blending='additive', colormap='gray');

    
viewer.add_points(seeds, size=5, name='seeds', opacity=1, face_color='violet', blending='opaque')

@viewer.bind_key('k')
def zplus(viewer):
    current_z = viewer.dims.current_step[1]
    viewer.dims.set_point(1, current_z+1)

@viewer.bind_key('j')
def zminus(viewer):
    current_z = viewer.dims.current_step[1]
    viewer.dims.set_point(1, current_z-1)


@viewer.bind_key('.')
def tplus(viewer):
    current_tp = viewer.dims.current_step[0]
    viewer.dims.set_point(0, current_tp + 1)


@viewer.bind_key(',')
def tminus(viewer):
    current_tp = viewer.dims.current_step[0]
    viewer.dims.set_point(0, current_tp - 1)


@magicgui(call_button='saveSeeds')
def saveSeeds():

    seeds = viewer.layers['seeds'].data       
    np.save(path + '\\seg\\seeds\\array\\seeds_array.npy', seeds) 
    
    f = open(path + "\\seg\\seeds\\for_biowulf\\seg_fixed.swarm", "a")
    for i in range(0, total_file_nums):    
        myString = 'python3 segment_round2.py ' + str(i)
        f.write(myString + '\n')

    f.close()
 
    
viewer.window.add_dock_widget(saveSeeds, area='right')



In [ ]:
#segment changed fromes

seeds = np.load(path + '\\seg\\seeds\\array\\seeds_array.npy')

for i in range(0, total_file_nums):
    if DDN.Utils.checkDifferenceBetweenSeedArrays(i, path):
        DDN.Segmentation.SegmentFromSeedArray(path, shape, i, seeds, save=True)
        
        
